In [1]:
import re
import pandas as pd
import numpy as np

In [26]:
with open('2020_rankings.txt') as myFile:
    text = myFile.read()

In [3]:
indeces = []
for i in range(len(text)):
    if text[i:i+7] == 'profile' and text[i+10]!='#':
        indeces.append(i-1)
indeces = indeces[8:]

starting_indeces = []
for item in indeces:
    for j in range(1,40):
        if (text[item-j].isalpha() == False) and (text[item-j] != '-'):
            starting_indeces.append(j)
            break

names_and_schools = []
for i in range(len(starting_indeces)):
    names_and_schools.append(text[indeces[i]-starting_indeces[i]+1:indeces[i]])

names = []
schools = []
for i in range(len(names_and_schools)):
    if i%2 == 0:
        names.append(names_and_schools[i])
    else:
        schools.append(names_and_schools[i])

In [4]:
for i in range(len(names)):
    first_name = names[i].split('-')[-1]
    last_name = '-'.join(names[i].split('-')[:-1])
    names[i] = f'{first_name} {last_name}'

In [5]:
ending_indeces = []
records = []
for i in range(len(text)):
    if text[i:i+7] == '<small>':
        for j in range(1,10):
            if text[i+9+j] == '|':
                records.append(text[i+9:i+9+j])

In [6]:
EligibilityYear_num = []
for i in range(len(text)):
    if text[i:i+8] == '</small>' and text[i-1].isnumeric() == False:
        EligibilityYear_num.append(text[i-2:i])
EligibilityYear_num = EligibilityYear_num[9:-26]

for i in range(len(EligibilityYear_num)):
    if EligibilityYear_num[i] == 'FR':
        EligibilityYear_num[i] = 0
    elif EligibilityYear_num[i] == 'SO':
        EligibilityYear_num[i] = 1
    elif EligibilityYear_num[i] == 'JR':
        EligibilityYear_num[i] = 2
    else:
        EligibilityYear_num[i] = 3
        
        
        
        

In [7]:
wins = []
losses = []
for i in range(len(records)):
    wins.append(records[i].split('-')[0])
    losses.append(records[i].split('-')[1])

In [8]:
ewb = [] #elo points, win %, bonus %
for i in range(len(text)):
    if text[i:i+8] == '<strong>':
        for j in range(1,10):
            if text[i+8+j] == '<':
                ewb.append(text[i+8:i+8+j])

In [9]:
for i in range(len(ewb)):
    ewb[i] = ewb[i].replace('%','')

In [10]:
elo_points = []
win_percentage = []
bonus_percentage = []
for i in range(len(ewb)):
    if i%3 == 0:
        elo_points.append(ewb[i])
    elif i%3 == 1:
        win_percentage.append(ewb[i])
    else:
            bonus_percentage.append(ewb[i])

In [11]:
elo_points = [float(item) for item in elo_points]
win_percentage = [float(item) for item in win_percentage]
bonus_percentage = [float(item) for item in bonus_percentage]

In [12]:
ranks = []
for i in range(len(names)):
    ranks.append(i%33+1)

In [13]:
conferences = []
for i in range(len(text)):
    if text[i:i+7] == '</a> | ':
        for j in range(1,10):
            if text[i+7+j] == '<':
                conferences.append(text[i+7:i+7+j])

In [14]:
cols = ['NameLower','Rank','WinPercentage','BonusPercentage','Wins','Losses','EloPoints','Conference','EligibilityYear_num']

In [15]:
df = pd.DataFrame(data=list(zip(names,ranks,win_percentage,bonus_percentage,wins,losses,elo_points,conferences,EligibilityYear_num)),columns=cols)

In [16]:
df

,NameLower,Rank,WinPercentage,BonusPercentage,Wins,Losses,EloPoints,Conference,EligibilityYear_num
0,spencer lee,1,100.00,88.89,18,0,1637.11,Big Ten,2
1,nick piccininni,2,92.86,76.92,26,2,1580.37,Big 12,3
2,jack mueller,3,92.86,76.92,13,1,1579.40,ACC,3
3,patrick glory,4,100.00,70.83,24,0,1561.11,EIWA,1
4,devin schroder,5,83.87,57.69,26,5,1474.04,Big Ten,2
...,...,...,...,...,...,...,...,...,...
325,ryan cloud,29,69.23,44.44,27,12,1373.48,MAC,3
326,jordan earnest,30,67.65,26.09,23,11,1372.26,MAC,0
327,zachary knighton-ward,31,62.16,43.48,23,14,1366.33,EIWA,1
328,luke luffman,32,60.71,29.41,17,11,1365.08,Big Ten,0


In [17]:
conferences_series = pd.Series(df['Conference'].unique())

In [18]:
for i in range(0,7):
    df[conferences_series[i]] = np.zeros(len(df['Conference']))
    for j in range(0,len(df['Conference'])-1):
        if df['Conference'][j] == conferences_series[i]:
            df[conferences_series[i]][j] = 1
        

C:\Users\fq1228hj\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
df.drop(labels='Conference',axis=1,inplace=True)

In [20]:
df = df.set_index('NameLower')

In [21]:
df['EWL'] = np.zeros(len(df))

In [22]:
df = df[['Rank', 'WinPercentage', 'BonusPercentage', 'Wins',
       'Losses', 'EloPoints', 'Big Ten', 'Big 12', 'ACC',
       'EIWA', 'PAC 12', 'MAC', 'EWL', 'SoCon', 'EligibilityYear_num']]

In [23]:
df

,Rank,WinPercentage,BonusPercentage,Wins,Losses,EloPoints,Big Ten,Big 12,ACC,EIWA,PAC 12,MAC,EWL,SoCon,EligibilityYear_num
NameLower,,,,,,,,,,,,,,,
spencer lee,1,100.00,88.89,18,0,1637.11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
nick piccininni,2,92.86,76.92,26,2,1580.37,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3
jack mueller,3,92.86,76.92,13,1,1579.40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3
patrick glory,4,100.00,70.83,24,0,1561.11,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
devin schroder,5,83.87,57.69,26,5,1474.04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ryan cloud,29,69.23,44.44,27,12,1373.48,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3
jordan earnest,30,67.65,26.09,23,11,1372.26,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
zachary knighton-ward,31,62.16,43.48,23,14,1366.33,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1


In [24]:
df.to_csv(r'C:\Users\fq1228hj\Documents\NCAA Wrestling\2020_test_data.csv')